In [ ]:
%load_ext nb_black
%load_ext autoreload
%autoreload 2

import os

print(os.getcwd())


def update_working_directory():
    from pathlib import Path

    p = Path(os.getcwd()).parents[0]
    os.chdir(p)
    print(p)


update_working_directory()

In [ ]:
path_dataset_test = "data/raw/20201009/dataset_test.pkl"

# Import

In [ ]:
import dill
import numpy as np
import pandas as pd
import plotly.graph_objects as go

pd.set_option("display.max_columns", None)

import src.models.performance_metrics as performance_metrics


# Dataset

In [ ]:
with open(path_dataset_test, "rb") as input_file:
    dataset_test_original = dill.load(input_file)

# Initialisation

In [ ]:
results_test_all_models = pd.DataFrame(columns=["metric"])
fig_precision_recall_curve = go.Figure()
fig_roc_auc_curve = go.Figure()

In [ ]:
list_models_to_compare = [
    "logistic_regression__20201014",
    "logistic_regression_mle__20201014",
]

# Getting data

In [ ]:
for model_name in list_models_to_compare:

    dataset_test = dataset_test_original.copy()

    with open(f"models/{model_name}__model.pkl", "rb") as input_file:
        model = dill.load(input_file)

    y_test = dataset_test[model.vardict["target"]].copy()
    dataset_test = model.preprocessing_inference(dataset_test)
    predictions = model.predict(dataset=dataset_test, target_present=False)
    predictions["y_true"] = y_test.values.tolist()

    binary_classification_results = (
        performance_metrics.get_binary_classification_results(
            predictions, model_name=f"{model_name}_test"
        )
    )

    regression_results = performance_metrics.get_regression_results(
        predictions, model_name=f"{model_name}_test"
    )

    results_test_model = {**binary_classification_results, **regression_results}

    results_test_model_table = pd.DataFrame.from_dict(
        results_test_model, orient="index", columns=["value"]
    )

    results_test_model_table.reset_index(inplace=True)
    results_test_model_table.rename(
        columns={"index": "metric", "value": model_name}, inplace=True
    )

    results_test_all_models = pd.merge(
        results_test_all_models,
        results_test_model_table,
        on="metric",
        how="outer",
    )

    # Create traces

    fig_precision_recall_curve = performance_metrics.add_precision_recall_curve(
        fig_precision_recall_curve, predictions, model_name
    )

    fig_roc_auc_curve = performance_metrics.add_roc_auc_curve(
        fig_roc_auc_curve, predictions, model_name
    )

# Analysis

## Model metrics

In [ ]:
results_test_all_models

## ROC AUC curve

In [ ]:
fig_roc_auc_curve.add_trace(
    go.Scatter(
        x=[0, 1],
        y=[0, 1],
        mode="lines",
        name="random",
        line=dict(color="black", dash="dash"),
    )
)

fig_roc_auc_curve = performance_metrics.add_square(
    fig_roc_auc_curve, x0=0, x1=1, y0=0, y1=1
)

fig_roc_auc_curve.update_layout(
    title="Receiver operating characteristic (ROC) curve",
    legend={"itemsizing": "constant"},
)

fig_roc_auc_curve.update_xaxes(title_text="False Positive Rate", range=[-0.05, 1.05])
fig_roc_auc_curve.update_yaxes(title_text="True Positive Rate", range=[-0.05, 1.05])

fig_roc_auc_curve.show()

## Precision - Recall curve

In [ ]:
fig_precision_recall_curve = performance_metrics.add_square(
    fig_precision_recall_curve, x0=0, x1=1, y0=0, y1=1
)

fig_precision_recall_curve.add_trace(
    go.Scatter(
        x=[0, 1],
        y=[
            binary_classification_results["random_precision"],
            binary_classification_results["random_precision"],
        ],
        mode="lines",
        name="Random precision",
        line=dict(color="black", dash="dash"),
    )
)

fig_precision_recall_curve.update_layout(
    title="Precision-Recall curve",
    legend={"itemsizing": "constant"},
)

fig_precision_recall_curve.update_xaxes(title_text="Recall", range=[-0.05, 1.05])
fig_precision_recall_curve.update_yaxes(title_text="Precision", range=[-0.05, 1.05])

fig_precision_recall_curve.show()